In [ ]:
import ast

class GreenCodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        # Stores results of the analysis
        self.issues = []

    def visit_For(self, node):
        # Check for nested loops which can be inefficient
        if any(isinstance(child, ast.For) for child in ast.iter_child_nodes(node)):
            self.issues.append({
                'issue': 'Nested loop detected',
                'lineno': node.lineno,
                'recommendation': 'Consider optimizing nested loops to reduce computational overhead'
            })
        self.generic_visit(node)

    def visit_While(self, node):
        # Check for potential infinite loops or high-complexity while loops
        if not self.is_simple_loop(node):
            self.issues.append({
                'issue': 'Complex or potential infinite while loop',
                'lineno': node.lineno,
                'recommendation': 'Ensure loop termination conditions are met and optimize for efficiency'
            })
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        # Check for redundant calculations inside functions
        if self.has_redundant_calculations(node):
            self.issues.append({
                'issue': 'Redundant calculations inside function',
                'lineno': node.lineno,
                'recommendation': 'Store repeated calculations in variables to reduce redundant computations'
            })
        self.generic_visit(node)

    def visit_Compare(self, node):
        # Check for repeated comparisons that could be stored in variables
        if self.is_redundant_comparison(node):
            self.issues.append({
                'issue': 'Redundant comparison detected',
                'lineno': node.lineno,
                'recommendation': 'Store the comparison in a variable if used repeatedly'
            })
        self.generic_visit(node)

    def is_simple_loop(self, node):
        # Basic check for simple loop termination (e.g., finite iterations)
        return any(isinstance(child, ast.Break) for child in ast.walk(node))

    def has_redundant_calculations(self, node):
        # Identify redundant calculations (e.g., repeated arithmetic operations)
        # For simplicity, detect if the same arithmetic operation appears more than once
        operations = [n for n in ast.walk(node) if isinstance(n, ast.BinOp)]
        return len(operations) > len(set(operations))

    def is_redundant_comparison(self, node):
        # Check if the comparison is repeated (e.g., `x > 5` multiple times)
        comparisons = [n for n in ast.walk(node) if isinstance(n, ast.Compare)]
        return len(comparisons) > len(set(comparisons))

    def analyze(self, code):
        # Parse the code using AST
        tree = ast.parse(code)
        # Visit each node in the AST
        self.visit(tree)
        return self.issues

def main():
    # Sample code to analyze
    sample_code = """
for i in range(10):
    for j in range(10):
        print(i, j)

x = 0
while True:
    x += 1
    if x > 1000:
        break

def example_function():
    for i in range(10):
        print(i * 2)
        print(i * 2)  # Redundant calculation
    if x > 5 and x > 5:  # Redundant comparison
        print('x is greater than 5')
"""

    # Initialize the analyzer
    analyzer = GreenCodeAnalyzer()
    # Analyze the sample code
    issues = analyzer.analyze(sample_code)

    # Display the analysis results
    if issues:
        print("Analysis Results:")
        for issue in issues:
            print(f"Line {issue['lineno']}: {issue['issue']}")
            print(f"Recommendation: {issue['recommendation']}\n")
    else:
        print("No issues detected. The code appears to be optimized.")

if __name__ == "__main__":
    main()


Analysis Results:
Line 2: Nested loop detected
Recommendation: Consider optimizing nested loops to reduce computational overhead



In [ ]:
import ast
import time

class EnhancedGreenCodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        # Stores results of the analysis
        self.issues = []
        # Constants for carbon footprint estimation (adjust as necessary)
        self.cpu_energy_per_cycle = 3.3e-11  # Energy per CPU cycle in joules
        self.co2_per_joule = 0.0000006  # CO2 produced per joule in kg

    def visit_For(self, node):
        # Check for nested loops which can be inefficient
        if any(isinstance(child, ast.For) for child in ast.iter_child_nodes(node)):
            self.issues.append({
                'issue': 'Nested loop detected',
                'lineno': node.lineno,
                'recommendation': 'Consider optimizing nested loops to reduce computational overhead',
                'cpu_cycles_estimation': self.estimate_cpu_cycles(node),
                'carbon_footprint_estimation': self.estimate_carbon_footprint(node)
            })
        self.generic_visit(node)

    def visit_While(self, node):
        # Check for potential infinite loops or high-complexity while loops
        if not self.is_simple_loop(node):
            self.issues.append({
                'issue': 'Complex or potential infinite while loop',
                'lineno': node.lineno,
                'recommendation': 'Ensure loop termination conditions are met and optimize for efficiency',
                'cpu_cycles_estimation': self.estimate_cpu_cycles(node),
                'carbon_footprint_estimation': self.estimate_carbon_footprint(node)
            })
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        # Check for redundant calculations inside functions
        if self.has_redundant_calculations(node):
            self.issues.append({
                'issue': 'Redundant calculations inside function',
                'lineno': node.lineno,
                'recommendation': 'Store repeated calculations in variables to reduce redundant computations',
                'cpu_cycles_estimation': self.estimate_cpu_cycles(node),
                'carbon_footprint_estimation': self.estimate_carbon_footprint(node)
            })
        self.generic_visit(node)

    def estimate_cpu_cycles(self, node):
        # Basic estimation of CPU cycles - Here we assume each operation takes a fixed number of cycles
        # This is a simple heuristic; a real estimation would be more complex
        operation_count = sum(isinstance(n, (ast.BinOp, ast.Call, ast.Compare, ast.For, ast.While)) for n in ast.walk(node))
        cpu_cycles = operation_count * 1000  # Assume 1000 cycles per operation for simplicity
        return cpu_cycles

    def estimate_carbon_footprint(self, node):
        # Estimate carbon footprint based on CPU cycles
        cpu_cycles = self.estimate_cpu_cycles(node)
        energy_consumed = cpu_cycles * self.cpu_energy_per_cycle  # in joules
        carbon_footprint = energy_consumed * self.co2_per_joule  # in kg CO2
        return carbon_footprint

    def is_simple_loop(self, node):
        # Basic check for simple loop termination (e.g., finite iterations)
        return any(isinstance(child, ast.Break) for child in ast.walk(node))

    def has_redundant_calculations(self, node):
        # Identify redundant calculations (e.g., repeated arithmetic operations)
        operations = [n for n in ast.walk(node) if isinstance(n, ast.BinOp)]
        return len(operations) > len(set(operations))

    def analyze(self, code):
        # Parse the code using AST
        tree = ast.parse(code)
        # Visit each node in the AST
        self.visit(tree)
        return self.issues

def main():
    # Sample code to analyze
    sample_code = """
for i in range(10):
    for j in range(10):
        print(i, j)

x = 0
while True:
    x += 1
    if x > 1000:
        break

def example_function():
    for i in range(10):
        print(i * 2)
        print(i * 2)  # Redundant calculation
    if x > 5 and x > 5:  # Redundant comparison
        print('x is greater than 5')
"""

    # Initialize the analyzer
    analyzer = EnhancedGreenCodeAnalyzer()
    # Analyze the sample code
    issues = analyzer.analyze(sample_code)

    # Display the analysis results
    if issues:
        print("Analysis Results:")
        for issue in issues:
            print(f"Line {issue['lineno']}: {issue['issue']}")
            print(f"Recommendation: {issue['recommendation']}")
            print(f"Estimated CPU Cycles: {issue['cpu_cycles_estimation']}")
            print(f"Estimated Carbon Footprint: {issue['carbon_footprint_estimation']:.10f} kg CO2\n")
    else:
        print("No issues detected. The code appears to be optimized.")

if __name__ == "__main__":
    main()


Analysis Results:
Line 2: Nested loop detected
Recommendation: Consider optimizing nested loops to reduce computational overhead
Estimated CPU Cycles: 5000
Estimated Carbon Footprint: 0.0000000000 kg CO2



In [ ]:
import ast

class EnhancedGreenCodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        # Stores results of the analysis
        self.issues = []
        self.corrections = []
        # Constants for carbon footprint estimation (adjust as necessary)
        self.cpu_energy_per_cycle = 3.3e-11  # Energy per CPU cycle in joules
        self.co2_per_joule = 0.0000006  # CO2 produced per joule in kg

    def visit_For(self, node):
        # Check for nested loops which can be inefficient
        if any(isinstance(child, ast.For) for child in ast.iter_child_nodes(node)):
            cpu_cycles = self.estimate_cpu_cycles(node)
            carbon_footprint = self.estimate_carbon_footprint(cpu_cycles)
            self.issues.append({
                'issue': 'Nested loop detected',
                'lineno': node.lineno,
                'recommendation': 'Consider optimizing nested loops to reduce computational overhead',
                'cpu_cycles_estimation': cpu_cycles,
                'carbon_footprint_estimation': carbon_footprint
            })
            # Suggest flattening the nested loop or using a more efficient algorithm
            self.corrections.append({
                'lineno': node.lineno,
                'correction': self.optimize_nested_loop(node)
            })
        self.generic_visit(node)

    def estimate_cpu_cycles(self, node):
        # More granular estimation of CPU cycles
        operation_count = 0
        for n in ast.walk(node):
            if isinstance(n, (ast.BinOp, ast.Compare)):
                operation_count += 10  # Simple arithmetic operation
            elif isinstance(n, (ast.Call)):
                operation_count += 100  # Function call
            elif isinstance(n, (ast.For, ast.While)):
                operation_count += 1000  # Loops are more complex
        return operation_count

    def estimate_carbon_footprint(self, cpu_cycles):
        # Estimate carbon footprint based on CPU cycles
        energy_consumed = cpu_cycles * self.cpu_energy_per_cycle  # in joules
        carbon_footprint = energy_consumed * self.co2_per_joule  # in kg CO2
        return carbon_footprint

    def optimize_nested_loop(self, node):
        # Provide an optimization suggestion for nested loops
        # For simplicity, suggest using list comprehensions or itertools.product
        # This function is context-aware and would need real data for complex optimizations
        if isinstance(node, ast.For):
            return "Consider using itertools.product for nested loops if independent."
        return None

    def analyze(self, code):
        # Parse the code using AST
        tree = ast.parse(code)
        # Visit each node in the AST
        self.visit(tree)
        return self.issues, self.corrections

def main():
    # Sample code to analyze
    sample_code = """
for i in range(10):
    for j in range(10):
        print(i, j)
"""

    # Initialize the analyzer
    analyzer = EnhancedGreenCodeAnalyzer()
    # Analyze the sample code
    issues, corrections = analyzer.analyze(sample_code)

    # Display the analysis results
    if issues:
        print("Analysis Results:")
        for issue in issues:
            print(f"Line {issue['lineno']}: {issue['issue']}")
            print(f"Recommendation: {issue['recommendation']}")
            print(f"Estimated CPU Cycles: {issue['cpu_cycles_estimation']}")
            print(f"Estimated Carbon Footprint: {issue['carbon_footprint_estimation']:.10f} kg CO2\n")
    else:
        print("No issues detected. The code appears to be optimized.")

    # Display suggested corrections
    if corrections:
        print("Suggested Code Corrections:")
        for correction in corrections:
            if correction['correction']:
                print(f"Line {correction['lineno']}: {correction['correction']}")
    else:
        print("No corrections suggested.")

if __name__ == "__main__":
    main()

Analysis Results:
Line 2: Nested loop detected
Recommendation: Consider optimizing nested loops to reduce computational overhead
Estimated CPU Cycles: 2300
Estimated Carbon Footprint: 0.0000000000 kg CO2

Suggested Code Corrections:
Line 2: Consider using itertools.product for nested loops if independent.


**Nested Loops**

In [ ]:
import ast
import textwrap
import itertools

class EnhancedGreenCodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.analysis_results = []

    def analyze_code(self, code):
        # Parse the code into an Abstract Syntax Tree (AST)
        tree = ast.parse(code)
        self.visit(tree)  # Visit each node
        return self.analysis_results

    def visit_For(self, node):
        # Check for nested loops
        if any(isinstance(n, ast.For) for n in ast.iter_child_nodes(node)):
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Nested loop detected',
                'recommendation': 'Consider optimizing nested loops to reduce computational overhead',
                'optimized_code': self.optimize_nested_loop(node)
            })
        self.generic_visit(node)

    def get_source_segment(self, node):
        # Helper method to extract source code segment
        return ast.unparse(node)  # Available in Python 3.9+

    def optimize_nested_loop(self, node):
        # Provide an optimization suggestion for nested loops
        if isinstance(node, ast.For):
            # Extract the ranges and loop variables
            outer_loop_var = node.target.id
            outer_loop_range = self.get_source_segment(node.iter)
            inner_loop = next((n for n in ast.iter_child_nodes(node) if isinstance(n, ast.For)), None)
            if inner_loop:
                inner_loop_var = inner_loop.target.id
                inner_loop_range = self.get_source_segment(inner_loop.iter)
                # Generate the optimized code snippet using itertools.product
                optimized_code = textwrap.dedent(f"""
                import itertools
                for {outer_loop_var}, {inner_loop_var} in itertools.product({outer_loop_range}, {inner_loop_range}):
                    # Your optimized code here
                    result = {outer_loop_var} * {inner_loop_var}  # Example computation
                """)
                return optimized_code
        return None

    def estimate_cpu_cycles(self, code):
        # Estimate CPU cycles for the entire code block
        tree = ast.parse(code)
        total_cycles = 0
        for node in ast.walk(tree):
            if isinstance(node, ast.For):
                inner_loops = sum(1 for n in ast.iter_child_nodes(node) if isinstance(n, ast.For))
                if inner_loops:
                    total_cycles += 5000 * inner_loops  # Increased complexity for nested loops
                else:
                    total_cycles += 3000  # Outer loop complexity
            elif isinstance(node, ast.BinOp):  # Consider computation operations
                total_cycles += 500
            elif isinstance(node, ast.Assign):
                total_cycles += 100  # Assignment operations
        return total_cycles

    def estimate_carbon_footprint(self, cpu_cycles):
        # Simplified carbon footprint estimation (in kg CO2)
        return cpu_cycles * 0.000000000001

    def compare_code_metrics(self, original_code, optimized_code):
        # Compare CPU cycles and carbon footprint for the entire code blocks
        original_cycles = self.estimate_cpu_cycles(original_code)
        optimized_cycles = self.estimate_cpu_cycles(optimized_code)
        original_carbon_footprint = self.estimate_carbon_footprint(original_cycles)
        optimized_carbon_footprint = self.estimate_carbon_footprint(optimized_cycles)
        return {
            'original_cycles': original_cycles,
            'optimized_cycles': optimized_cycles,
            'original_carbon_footprint': original_carbon_footprint,
            'optimized_carbon_footprint': optimized_carbon_footprint
        }


def main():
    # Revised sample code with more inefficiencies for demonstration
    sample_code = textwrap.dedent("""
    result = 0
    for i in range(1000):  # Larger outer loop
        for j in range(1000):  # Larger inner loop
            for k in range(100):  # Additional nested loop
                result += i * j * k  # Computationally intensive operation
                for l in range(50):  # Deeply nested loop
                    result += i * j * k * l  # More intensive operation
    """)

    # Create an instance of the analyzer
    analyzer = EnhancedGreenCodeAnalyzer()

    # Analyze the code
    analysis_results = analyzer.analyze_code(sample_code)

    # Calculate metrics for original code
    original_cpu_cycles = analyzer.estimate_cpu_cycles(sample_code)
    original_carbon_footprint = analyzer.estimate_carbon_footprint(original_cpu_cycles)

    # Print analysis results and optimized code suggestions
    optimized_code = sample_code
    for result in analysis_results:
        print(f"Line {result['line_number']}: {result['issue']}")
        print(f"Recommendation: {result['recommendation']}")
        if result['optimized_code']:
            optimized_code = result['optimized_code']
            print("\nSuggested Code Corrections:")
            print(result['optimized_code'])

    # Calculate metrics for optimized code
    comparison_metrics = analyzer.compare_code_metrics(sample_code, optimized_code)
    optimized_cpu_cycles = comparison_metrics['optimized_cycles']
    optimized_carbon_footprint = comparison_metrics['optimized_carbon_footprint']

    # Output the comparison
    print("\nComparison of CPU Cycles and Carbon Footprint for the entire code:")
    print(f"Original Code CPU Cycles: {original_cpu_cycles}")
    print(f"Optimized Code CPU Cycles: {optimized_cpu_cycles}")
    print(f"Original Code Carbon Footprint: {original_carbon_footprint:.10f} kg CO2")
    print(f"Optimized Code Carbon Footprint: {optimized_carbon_footprint:.10f} kg CO2")

if __name__ == "__main__":
    main()


Line 3: Nested loop detected
Recommendation: Consider optimizing nested loops to reduce computational overhead

Suggested Code Corrections:

import itertools
for i, j in itertools.product(range(1000), range(1000)):
    # Your optimized code here
    result = i * j  # Example computation

Line 4: Nested loop detected
Recommendation: Consider optimizing nested loops to reduce computational overhead

Suggested Code Corrections:

import itertools
for j, k in itertools.product(range(1000), range(100)):
    # Your optimized code here
    result = j * k  # Example computation

Line 5: Nested loop detected
Recommendation: Consider optimizing nested loops to reduce computational overhead

Suggested Code Corrections:

import itertools
for k, l in itertools.product(range(100), range(50)):
    # Your optimized code here
    result = k * l  # Example computation


Comparison of CPU Cycles and Carbon Footprint for the entire code:
Original Code CPU Cycles: 20600
Optimized Code CPU Cycles: 3600
Origi

 2.Testing on Difference code Smell : Inefficient List Type

In [ ]:
import ast
import textwrap
import itertools

class EnhancedGreenCodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.analysis_results = []

    def analyze_code(self, code):
        # Parse the code into an Abstract Syntax Tree (AST)
        tree = ast.parse(code)
        self.visit(tree)  # Visit each node
        return self.analysis_results

    def visit_For(self, node):
        # Check for list operations inside loops
        if any(isinstance(n, ast.Call) and isinstance(n.func, ast.Attribute) and n.func.attr == 'append'
               for n in ast.walk(node)):
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Inefficient list operations detected',
                'recommendation': 'Consider using list comprehensions or pre-allocating lists for better performance',
                'optimized_code': self.optimize_list_operations(node)
            })
        self.generic_visit(node)

    def get_source_segment(self, node):
        # Helper method to extract source code segment
        return ast.unparse(node)  # Available in Python 3.9+

    def optimize_list_operations(self, node):
        # Provide an optimization suggestion for list operations
        if isinstance(node, ast.For):
            # Extract loop variables and operations
            loop_var = node.target.id
            loop_range = self.get_source_segment(node.iter)
            operations = [n for n in ast.walk(node) if isinstance(n, ast.Call) and isinstance(n.func, ast.Attribute) and n.func.attr == 'append']
            if operations:
                # Generate the optimized code snippet
                optimized_code = textwrap.dedent(f"""
                result = []
                for {loop_var} in {loop_range}:  # Pre-allocate list
                    # Perform operations directly
                    result.append({loop_var} * 2)  # Example operation
                """)
                return optimized_code
        return None

    def estimate_cpu_cycles(self, code):
        # Estimate CPU cycles for the entire code block
        tree = ast.parse(code)
        total_cycles = 0
        for node in ast.walk(tree):
            if isinstance(node, ast.For):
                total_cycles += 3000  # Loop complexity
            elif isinstance(node, ast.Call) and isinstance(node.func, ast.Attribute) and node.func.attr == 'append':
                total_cycles += 1000  # Append operations
            elif isinstance(node, ast.BinOp):  # Consider computation operations
                total_cycles += 500
            elif isinstance(node, ast.Assign):
                total_cycles += 100  # Assignment operations
        return total_cycles

    def estimate_carbon_footprint(self, cpu_cycles):
        # Simplified carbon footprint estimation (in kg CO2)
        return cpu_cycles * 0.000000000001

    def compare_code_metrics(self, original_code, optimized_code):
        # Compare CPU cycles and carbon footprint for the entire code blocks
        original_cycles = self.estimate_cpu_cycles(original_code)
        optimized_cycles = self.estimate_cpu_cycles(optimized_code)
        original_carbon_footprint = self.estimate_carbon_footprint(original_cycles)
        optimized_carbon_footprint = self.estimate_carbon_footprint(optimized_cycles)
        return {
            'original_cycles': original_cycles,
            'optimized_cycles': optimized_cycles,
            'original_carbon_footprint': original_carbon_footprint,
            'optimized_carbon_footprint': optimized_carbon_footprint
        }


def main():
    # Revised sample code with inefficient list operations for demonstration
    sample_code = textwrap.dedent("""
    result = []
    for i in range(1000):
        for j in range(100):
            result.append(i + j)  # Inefficient list operation inside nested loops
            result.append(i * j)  # Additional inefficient operation
    """)

    # Create an instance of the analyzer
    analyzer = EnhancedGreenCodeAnalyzer()

    # Analyze the code
    analysis_results = analyzer.analyze_code(sample_code)

    # Calculate metrics for original code
    original_cpu_cycles = analyzer.estimate_cpu_cycles(sample_code)
    original_carbon_footprint = analyzer.estimate_carbon_footprint(original_cpu_cycles)

    # Print analysis results and optimized code suggestions
    optimized_code = sample_code
    for result in analysis_results:
        print(f"Line {result['line_number']}: {result['issue']}")
        print(f"Recommendation: {result['recommendation']}")
        if result['optimized_code']:
            optimized_code = result['optimized_code']
            print("\nSuggested Code Corrections:")
            print(result['optimized_code'])

    # Calculate metrics for optimized code
    comparison_metrics = analyzer.compare_code_metrics(sample_code, optimized_code)
    optimized_cpu_cycles = comparison_metrics['optimized_cycles']
    optimized_carbon_footprint = comparison_metrics['optimized_carbon_footprint']

    # Output the comparison
    print("\nComparison of CPU Cycles and Carbon Footprint for the entire code:")
    print(f"Original Code CPU Cycles: {original_cpu_cycles}")
    print(f"Optimized Code CPU Cycles: {optimized_cpu_cycles}")
    print(f"Original Code Carbon Footprint: {original_carbon_footprint:.10f} kg CO2")
    print(f"Optimized Code Carbon Footprint: {optimized_carbon_footprint:.10f} kg CO2")

if __name__ == "__main__":
    main()


Line 3: Inefficient list operations detected
Recommendation: Consider using list comprehensions or pre-allocating lists for better performance

Suggested Code Corrections:

result = []
for i in range(1000):  # Pre-allocate list
    # Perform operations directly
    result.append(i * 2)  # Example operation

Line 4: Inefficient list operations detected
Recommendation: Consider using list comprehensions or pre-allocating lists for better performance

Suggested Code Corrections:

result = []
for j in range(100):  # Pre-allocate list
    # Perform operations directly
    result.append(j * 2)  # Example operation


Comparison of CPU Cycles and Carbon Footprint for the entire code:
Original Code CPU Cycles: 9100
Optimized Code CPU Cycles: 4600
Original Code Carbon Footprint: 0.0000000091 kg CO2
Optimized Code Carbon Footprint: 0.0000000046 kg CO2


**3. Inefficient/ Multiple Sorting and 4. Inefficent Algorithms :**

In [ ]:
import ast
import textwrap

class EnhancedGreenCodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.analysis_results = []

    def analyze_code(self, code):
        # Parse the code into an Abstract Syntax Tree (AST)
        tree = ast.parse(code)
        self.visit(tree)  # Visit each node
        return self.analysis_results

    def visit_FunctionDef(self, node):
        # Check for repetitive code
        self.check_repetitive_code(node)
        self.generic_visit(node)

    def check_repetitive_code(self, node):
        # Simple check for repetitive code
        # This is a basic example and may require more advanced analysis
        source_code = ast.unparse(node)
        if source_code.count('result += i * j') > 1:
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Repetitive code detected',
                'recommendation': 'Consider refactoring repetitive code into a function',
                'optimized_code': self.optimize_repetitive_code(node)
            })

    def optimize_repetitive_code(self, node):
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            # Generate the optimized code snippet with a helper function
            optimized_code = textwrap.dedent(f"""
            def compute_product(i, j):
                return i * j

            def {function_name}():
                result = 0
                for i in range(100):
                    for j in range(100):
                        result += compute_product(i, j)
                """)
            return optimized_code
        return None

    def visit_Call(self, node):
        # Check for inefficient algorithms, such as multiple sorts
        if isinstance(node.func, ast.Attribute) and node.func.attr == 'sort':
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Inefficient algorithm detected',
                'recommendation': 'Avoid sorting multiple times; sort once and use the result',
                'optimized_code': self.optimize_inefficient_algorithm(node)
            })
        self.generic_visit(node)

    def optimize_inefficient_algorithm(self, node):
        # Simple example to avoid multiple sorts
        optimized_code = textwrap.dedent("""
        data = [5, 3, 2, 8, 1]
        data.sort()
        # Use sorted data
        # Perform operations on sorted data
        """)
        return optimized_code

    def estimate_cpu_cycles(self, code):
        # Estimate CPU cycles for the entire code block
        tree = ast.parse(code)
        total_cycles = 0
        for node in ast.walk(tree):
            if isinstance(node, ast.For):
                inner_loops = sum(1 for n in ast.iter_child_nodes(node) if isinstance(n, ast.For))
                if inner_loops:
                    total_cycles += 5000 * inner_loops  # Increased complexity for nested loops
                else:
                    total_cycles += 3000  # Outer loop complexity
            elif isinstance(node, ast.BinOp):  # Consider computation operations
                total_cycles += 500
            elif isinstance(node, ast.Assign):
                total_cycles += 100  # Assignment operations
            elif isinstance(node, ast.FunctionDef):
                total_cycles += 2000  # Basic function overhead
        return total_cycles

    def estimate_carbon_footprint(self, cpu_cycles):
        # Simplified carbon footprint estimation (in kg CO2)
        return cpu_cycles * 0.000000000001

    def compare_code_metrics(self, original_code, optimized_code):
        # Compare CPU cycles and carbon footprint for the entire code blocks
        original_cycles = self.estimate_cpu_cycles(original_code)
        optimized_cycles = self.estimate_cpu_cycles(optimized_code)
        original_carbon_footprint = self.estimate_carbon_footprint(original_cycles)
        optimized_carbon_footprint = self.estimate_carbon_footprint(optimized_cycles)
        return {
            'original_cycles': original_cycles,
            'optimized_cycles': optimized_cycles,
            'original_carbon_footprint': original_carbon_footprint,
            'optimized_carbon_footprint': optimized_carbon_footprint
        }

def main():
    # Sample code with different code smells
    sample_code = textwrap.dedent("""
    def example_function():
        result = 0
        for i in range(100):
            for j in range(100):
                result += i * j
                result += i * j  # Repetitive code block
                result += i * j  # Repetitive code block
        data = [5, 3, 2, 8, 1]
        data.sort()
        data.sort()  # Inefficient sorting operation
    """)

    # Create an instance of the analyzer
    analyzer = EnhancedGreenCodeAnalyzer()

    # Analyze the code
    analysis_results = analyzer.analyze_code(sample_code)

    # Calculate metrics for original code
    original_cpu_cycles = analyzer.estimate_cpu_cycles(sample_code)
    original_carbon_footprint = analyzer.estimate_carbon_footprint(original_cpu_cycles)

    # Print analysis results and optimized code suggestions
    optimized_code = sample_code
    for result in analysis_results:
        print(f"Line {result['line_number']}: {result['issue']}")
        print(f"Recommendation: {result['recommendation']}")
        if result['optimized_code']:
            optimized_code = result['optimized_code']
            print("\nSuggested Code Corrections:")
            print(result['optimized_code'])

    # Calculate metrics for optimized code
    comparison_metrics = analyzer.compare_code_metrics(sample_code, optimized_code)
    optimized_cpu_cycles = comparison_metrics['optimized_cycles']
    optimized_carbon_footprint = comparison_metrics['optimized_carbon_footprint']

    # Output the comparison
    print("\nComparison of CPU Cycles and Carbon Footprint for the entire code:")
    print(f"Original Code CPU Cycles: {original_cpu_cycles}")
    print(f"Optimized Code CPU Cycles: {optimized_cpu_cycles}")
    print(f"Original Code Carbon Footprint: {original_carbon_footprint:.10f} kg CO2")
    print(f"Optimized Code Carbon Footprint: {optimized_carbon_footprint:.10f} kg CO2")

if __name__ == "__main__":
    main()


Line 2: Repetitive code detected
Recommendation: Consider refactoring repetitive code into a function

Suggested Code Corrections:

def compute_product(i, j):
    return i * j

def example_function():
    result = 0
    for i in range(100):
        for j in range(100):
            result += compute_product(i, j)

Line 10: Inefficient algorithm detected
Recommendation: Avoid sorting multiple times; sort once and use the result

Suggested Code Corrections:

data = [5, 3, 2, 8, 1]
data.sort()
# Use sorted data
# Perform operations on sorted data

Line 11: Inefficient algorithm detected
Recommendation: Avoid sorting multiple times; sort once and use the result

Suggested Code Corrections:

data = [5, 3, 2, 8, 1]
data.sort()
# Use sorted data
# Perform operations on sorted data


Comparison of CPU Cycles and Carbon Footprint for the entire code:
Original Code CPU Cycles: 11700
Optimized Code CPU Cycles: 100
Original Code Carbon Footprint: 0.0000000117 kg CO2
Optimized Code Carbon Footprint:

Organizing the 4 different code Smells into One

In [ ]:
import ast
import textwrap
import itertools

class EnhancedGreenCodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.analysis_results = []

    def analyze_code(self, code):
        # Parse the code into an Abstract Syntax Tree (AST)
        tree = ast.parse(code)
        self.visit(tree)  # Visit each node
        return self.analysis_results

    def visit_For(self, node):
        # Check for nested loops
        if any(isinstance(n, ast.For) for n in ast.iter_child_nodes(node)):
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Nested loop detected',
                'recommendation': 'Consider optimizing nested loops to reduce computational overhead',
                'optimized_code': self.optimize_nested_loop(node)
            })
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        # Check for repetitive code in functions
        code_blocks = self.get_function_code_blocks(node)
        repetitive_blocks = self.detect_repetitive_code(code_blocks)
        for block in repetitive_blocks:
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Repetitive code detected',
                'recommendation': 'Consider refactoring repetitive code into a function',
                'optimized_code': self.optimize_repetitive_code(block)
            })
        self.generic_visit(node)

    def visit_While(self, node):
        # Check for inefficient algorithms (e.g., while loops with high iteration count)
        if isinstance(node.test, ast.Compare) and isinstance(node.test.left, ast.Name) and isinstance(node.test.comparators[0], ast.Constant):
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Inefficient algorithm detected',
                'recommendation': 'Consider optimizing the while loop condition',
                'optimized_code': self.optimize_while_loop(node)
            })
        self.generic_visit(node)

    def visit_BinOp(self, node):
        # Check for redundant computations (e.g., unnecessary repeated operations)
        if isinstance(node.op, ast.Mult) and isinstance(node.left, ast.BinOp):
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Redundant computation detected',
                'recommendation': 'Consider optimizing redundant computations',
                'optimized_code': self.optimize_redundant_computation(node)
            })
        self.generic_visit(node)

    def get_source_segment(self, node):
        # Helper method to extract source code segment
        return ast.unparse(node)  # Available in Python 3.9+

    def get_function_code_blocks(self, node):
        # Extract code blocks from a function definition
        return [ast.unparse(n) for n in node.body if isinstance(n, ast.Expr)]

    def detect_repetitive_code(self, code_blocks):
        # Dummy implementation for detecting repetitive code
        return [block for block in code_blocks if code_blocks.count(block) > 1]

    def optimize_repetitive_code(self, block):
        # Dummy implementation for optimizing repetitive code
        return f"# Refactored code from repetitive block: {block}"

    def optimize_nested_loop(self, node):
        # Provide an optimization suggestion for nested loops
        if isinstance(node, ast.For):
            outer_loop_var = node.target.id
            outer_loop_range = self.get_source_segment(node.iter)
            inner_loop = next((n for n in ast.iter_child_nodes(node) if isinstance(n, ast.For)), None)
            if inner_loop:
                inner_loop_var = inner_loop.target.id
                inner_loop_range = self.get_source_segment(inner_loop.iter)
                optimized_code = textwrap.dedent(f"""
                import itertools
                for {outer_loop_var}, {inner_loop_var} in itertools.product({outer_loop_range}, {inner_loop_range}):
                    # Your optimized code here
                    pass  # Added pass statement to avoid empty code block
                """)
                return optimized_code
        return None

    def optimize_while_loop(self, node):
        # Provide an optimization suggestion for while loops
        return textwrap.dedent("""
        # Optimized while loop condition
        while <optimized condition>:
            # Your optimized code here
            pass
        """)

    def optimize_redundant_computation(self, node):
        # Provide an optimization suggestion for redundant computations
        return textwrap.dedent("""
        # Optimized redundant computation
        result = <optimized expression>
        """)

    def estimate_cpu_cycles(self, code):
        # Estimate CPU cycles for the entire code block
        tree = ast.parse(code)
        total_cycles = 0
        for node in ast.walk(tree):
            if isinstance(node, ast.For):
                inner_loops = sum(1 for n in ast.iter_child_nodes(node) if isinstance(n, ast.For))
                if inner_loops:
                    total_cycles += 5000 * inner_loops
                else:
                    total_cycles += 3000
            elif isinstance(node, ast.BinOp):
                total_cycles += 500
            elif isinstance(node, ast.Assign):
                total_cycles += 100
        return total_cycles

    def estimate_carbon_footprint(self, cpu_cycles):
        # Simplified carbon footprint estimation (in kg CO2)
        return cpu_cycles * 0.000000000001

    def compare_code_metrics(self, original_code, optimized_code):
        # Compare CPU cycles and carbon footprint for the entire code blocks
        original_cycles = self.estimate_cpu_cycles(original_code)
        optimized_cycles = self.estimate_cpu_cycles(optimized_code)
        original_carbon_footprint = self.estimate_carbon_footprint(original_cycles)
        optimized_carbon_footprint = self.estimate_carbon_footprint(optimized_cycles)
        return {
            'original_cycles': original_cycles,
            'optimized_cycles': optimized_cycles,
            'original_carbon_footprint': original_carbon_footprint,
            'optimized_carbon_footprint': optimized_carbon_footprint
        }

def main():
    # Get user input for code analysis
    print("Enter your Python code (end input with a blank line):")
    code_lines = []
    while True:
        line = input()
        if line:
            code_lines.append(line)
        else:
            break
    user_code = "\n".join(code_lines)

    # Create an instance of the analyzer
    analyzer = EnhancedGreenCodeAnalyzer()

    # Analyze the code
    analysis_results = analyzer.analyze_code(user_code)

    # Calculate metrics for original code
    original_cpu_cycles = analyzer.estimate_cpu_cycles(user_code)
    original_carbon_footprint = analyzer.estimate_carbon_footprint(original_cpu_cycles)

    # Print analysis results and optimized code suggestions
    optimized_code = user_code
    for result in analysis_results:
        print(f"\nLine {result['line_number']}: {result['issue']}")
        print(f"Recommendation: {result['recommendation']}")
        if result['optimized_code']:
            optimized_code = result['optimized_code']
            print("Suggested Code Corrections:")
            print(result['optimized_code'])

    # Calculate metrics for optimized code
    comparison_metrics = analyzer.compare_code_metrics(user_code, optimized_code)
    optimized_cpu_cycles = comparison_metrics['optimized_cycles']
    optimized_carbon_footprint = comparison_metrics['optimized_carbon_footprint']

    # Output the comparison
    print("\nComparison of CPU Cycles and Carbon Footprint for the entire code:")
    print(f"Original Code CPU Cycles: {original_cpu_cycles}")
    print(f"Optimized Code CPU Cycles: {optimized_cpu_cycles}")
    print(f"Original Code Carbon Footprint: {original_carbon_footprint:.10f} kg CO2")
    print(f"Optimized Code Carbon Footprint: {optimized_carbon_footprint:.10f} kg CO2")

if __name__ == "__main__":
    main()


Enter your Python code (end input with a blank line):
def example_function():         result = 0         for i in range(100):             for j in range(100):                 result += i * j                 result += i * j  # Repetitive code block                 result += i * j  # Repetitive code block         data = [5, 3, 2, 8, 1]         data.sort()         data.sort()
def example_function():         result = 0         for i in range(100):             for j in range(100):                 result += i * j                 result += i * j  # Repetitive code block                 result += i * j  # Repetitive code block         data = [5, 3, 2, 8, 1]         data.sort()         data.sort()
def example_function():         result = 0         for i in range(100):             for j in range(100):                 result += i * j                 result += i * j  # Repetitive code block                 result += i * j  # Repetitive code block         data = [5, 3, 2, 8, 1]         data.sort() 

KeyboardInterrupt: Interrupted by user

In [5]:
import ast
import textwrap
import itertools

class EnhancedGreenCodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.analysis_results = []

    def analyze_code(self, code):
        # Parse the code into an Abstract Syntax Tree (AST)
        tree = ast.parse(code)
        self.visit(tree)  # Visit each node in the tree
        return self.analysis_results

    def visit_For(self, node):
        # Check for nested loops
        if any(isinstance(n, ast.For) for n in ast.iter_child_nodes(node)):
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Nested loop detected',
                'recommendation': 'Consider optimizing nested loops to reduce computational overhead',
                'optimized_code': self.optimize_nested_loop(node)
            })
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        # Check for repetitive code in functions
        code_blocks = self.get_function_code_blocks(node)
        repetitive_blocks = self.detect_repetitive_code(code_blocks)
        if repetitive_blocks:
            for block in repetitive_blocks:
                self.analysis_results.append({
                    'line_number': node.lineno,
                    'issue': 'Repetitive code detected',
                    'recommendation': 'Consider refactoring repetitive code into a function',
                    'optimized_code': self.optimize_repetitive_code(block)
                })
        self.generic_visit(node)

    def visit_While(self, node):
        # Check for inefficient algorithms (e.g., while loops with high iteration count)
        if isinstance(node.test, ast.Compare) and isinstance(node.test.left, ast.Name) and isinstance(node.test.comparators[0], ast.Constant):
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Inefficient algorithm detected',
                'recommendation': 'Consider optimizing the while loop condition',
                'optimized_code': self.optimize_while_loop(node)
            })
        self.generic_visit(node)

    def visit_BinOp(self, node):
        # Check for redundant computations (e.g., unnecessary repeated operations)
        if isinstance(node.op, ast.Mult) and isinstance(node.left, ast.BinOp):
            self.analysis_results.append({
                'line_number': node.lineno,
                'issue': 'Redundant computation detected',
                'recommendation': 'Consider optimizing redundant computations',
                'optimized_code': self.optimize_redundant_computation(node)
            })
        self.generic_visit(node)

    def get_function_code_blocks(self, node):
        # Extract code blocks from a function definition
        return [ast.unparse(n) for n in node.body if isinstance(n, ast.Expr)]

    def detect_repetitive_code(self, code_blocks):
        # Detect repetitive code blocks
        return [block for block in code_blocks if code_blocks.count(block) > 1]

    def optimize_repetitive_code(self, block):
        # Provide a suggestion to refactor repetitive code
        return f"# Refactored code from repetitive block: {block}"

    def optimize_nested_loop(self, node):
        # Provide an optimization suggestion for nested loops
        if isinstance(node, ast.For):
            outer_loop_var = node.target.id
            outer_loop_range = ast.unparse(node.iter)
            inner_loop = next((n for n in ast.iter_child_nodes(node) if isinstance(n, ast.For)), None)
            if inner_loop:
                inner_loop_var = inner_loop.target.id
                inner_loop_range = ast.unparse(inner_loop.iter)
                optimized_code = textwrap.dedent(f"""
                import itertools
                for {outer_loop_var}, {inner_loop_var} in itertools.product({outer_loop_range}, {inner_loop_range}):
                    # Your optimized code here
                    pass
                """)
                return optimized_code
        return None

    def optimize_while_loop(self, node):
        # Provide an optimization suggestion for while loops
        return textwrap.dedent("""
        # Optimized while loop condition
        while <optimized condition>:
            # Your optimized code here
            pass
        """)

    def optimize_redundant_computation(self, node):
        # Provide an optimization suggestion for redundant computations
        return textwrap.dedent("""
        # Optimized redundant computation
        result = <optimized expression>
        """)

    def estimate_cpu_cycles(self, code):
        # Estimate CPU cycles for the entire code block
        tree = ast.parse(code)
        total_cycles = 0
        for node in ast.walk(tree):
            if isinstance(node, ast.For):
                inner_loops = sum(1 for n in ast.iter_child_nodes(node) if isinstance(n, ast.For))
                if inner_loops:
                    total_cycles += 5000 * inner_loops
                else:
                    total_cycles += 3000
            elif isinstance(node, ast.BinOp):
                total_cycles += 500
            elif isinstance(node, ast.Assign):
                total_cycles += 100
        return total_cycles

    def estimate_carbon_footprint(self, cpu_cycles):
        # Simplified carbon footprint estimation (in kg CO2)
        return cpu_cycles * 0.000000000001

    def compare_code_metrics(self, original_code, optimized_code):
        # Compare CPU cycles and carbon footprint for the entire code blocks
        original_cycles = self.estimate_cpu_cycles(original_code)
        optimized_cycles = self.estimate_cpu_cycles(optimized_code)
        original_carbon_footprint = self.estimate_carbon_footprint(original_cycles)
        optimized_carbon_footprint = self.estimate_carbon_footprint(optimized_cycles)
        return {
            'original_cycles': original_cycles,
            'optimized_cycles': optimized_cycles,
            'original_carbon_footprint': original_carbon_footprint,
            'optimized_carbon_footprint': optimized_carbon_footprint
        }

# Replace the input() function for pre-defined testing or simulate user input
def main():
    print("Enter user code:")  # Prompt for user code
    user_code = '''
def calculate_matrix_multiplication(matrix_a, matrix_b):
    result = [[0 for _ in range(len(matrix_b[0]))] for _ in range(len(matrix_a))]
    for i in range(len(matrix_a)):
        for j in range(len(matrix_b[0])):
            for k in range(len(matrix_b)):
                result[i][j] += matrix_a[i][k] * matrix_b[k][j]
    return result
'''

    print("Enter optimized code:")  # Prompt for optimized code
    optimized_code = '''
def calculate_matrix_multiplication(matrix_a, matrix_b):
    return [[sum(a * b for a, b in zip(row_a, col_b)) for col_b in zip(*matrix_b)] for row_a in matrix_a]
'''

    # Now continue with the metrics calculation using the analyzer
    analyzer = EnhancedGreenCodeAnalyzer() # Use the correct class name
    comparison_metrics = analyzer.compare_code_metrics(user_code, optimized_code)

    # Output the comparison
    print("\nComparison of CPU Cycles and Carbon Footprint:")
    print(f"Original Code CPU Cycles: {comparison_metrics['original_cycles']}")
    print(f"Optimized Code CPU Cycles: {comparison_metrics['optimized_cycles']}")
    print(f"Original Carbon Footprint: {comparison_metrics['original_carbon_footprint']:.10f} kg CO2")
    print(f"Optimized Carbon Footprint: {comparison_metrics['optimized_carbon_footprint']:.10f} kg CO2")

# Call the main function
if __name__ == "__main__":
    main()


Enter user code:
Enter optimized code:

Comparison of CPU Cycles and Carbon Footprint:
Original Code CPU Cycles: 13600
Optimized Code CPU Cycles: 500
Original Carbon Footprint: 0.0000000136 kg CO2
Optimized Carbon Footprint: 0.0000000005 kg CO2
